## Auto gen Example Tool Use 

In [1]:
#use requirements_e7.txt to install the required packages

In [8]:
import os
import dotenv
from autogen import ConversableAgent
import openai

dotenv.load_dotenv("..\\common\\credentials.env", verbose=True, override=True)

True

Default config for autogen points to open ai format APIs. 
To use azure API's set config as below

In [9]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
aoai_api_type = os.environ["OPENAI_API_TYPE"]
deployment_name = "gpt-35-turbo" # "gpt-4" #use GPT-4 as 3.5 turbo seems to not work well for math problems
aoai_api_version = "2024-02-15-preview" #Using latest version as of this date. Older versions may not work with the code as syntax has changed drastically.

config_list = [
    {
        "model": deployment_name,  
        "base_url": aoai_endpoint, 
        "api_type": aoai_api_type, 
        "api_version": aoai_api_version, 
        "api_key": aoai_api_key,
        "temperature": 0.7
 }
]

In [10]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

In [11]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> int>

In [12]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

c:\Users\nileshjoshi\git\aoai-poc-hub\common\envs\aoai_poc_e4\Lib\site-packages\autogen\agentchat\conversable_agent.py:2573: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
c:\Users\nileshjoshi\git\aoai-poc-hub\common\envs\aoai_poc_e4\Lib\site-packages\autogen\agentchat\conversable_agent.py:2492: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


Always provide a clear and concise description for the tool as it helps the agent's underlying LLM to understand the tool's usage.

In [13]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_PcGCYce58Z2l7TOYift6Hf5D): calculator *****
Arguments: 
{"a": 44232, "b": 13312, "operator": "/"}
***************************************************************************
***** Suggested tool call (call_wLcanj7NUc8JEMqBEcovFK2i): calculator *****
Arguments: 
{"a": 232, "b": 32, "operator": "-"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_PcGCYce58Z2l7TOYift6Hf5D) *****
3
**********************************************************************

--------------------------------------

let us verify the answer

In [14]:
(44232 + int(13312 / (232 - 32))) * 5

221490

The answer is correct. You can see that the assistant is able to understand the tool’s usage and perform calculation correctly.